# This is where we calibrate our camera

First we load our chessboard images and use OpenCV2 to find the location of the corners on the image. 

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

shape = None
chessboard_corner_images = []

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    shape = gray.shape[::-1]
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        chessboard_corner_images.append(img)

 We draw lines between the corners and display them just to verify that the correct corners were detected. See below.

In [ ]:
test_images = [0,2,5,7]

plt.figure(figsize=(12,10))
for plot_num, i in enumerate(test_images):
    plt.subplot(2,2,(plot_num + 1))
    plt.imshow(chessboard_corner_images[i])

Here we calculate the camera distortion matrix and the calibration matrix.  We use the OpenCV2 calibrateCamera function to do do this.  This function takes the locations of the previously detected corners on the chessboard.

In [ ]:
%matplotlib inline
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)


Here we view some undistorted images using the previously calculated matrices.

In [ ]:
plt.imshow(cv2.imread(images[3]))

In [ ]:
dst = cv2.undistort(cv2.imread(images[3]), mtx, dist, None, mtx)
plt.imshow(dst)

Here we view an undistorted image with straight lane lines.  We manually select 4 points to use for the perspective transform.

In [ ]:
test_images = glob.glob('./test_images/*.jpg')

fig = plt.figure(figsize=(15,10))

img = plt.imread(test_images[6])
undist = cv2.undistort(img, mtx, dist, None, mtx)
plt.imshow(undist)

From the image above we choose the following points and calculate the perspective transform matrix.

In [ ]:
orig_points = np.float32([[210,725],[590,450],[685,450],[1110,725]])
dest_points = np.float32([[210,725],[210,0],[1110,0],[1110,725]])

M = cv2.getPerspectiveTransform(orig_points, dest_points)
Minv = cv2.getPerspectiveTransform(dest_points, orig_points)

Here we view the image after the perspective transform is applied to verify that the lines are vertical and parallel.

In [ ]:
img = plt.imread(test_images[6])
undist = cv2.undistort(img, mtx, dist, None, mtx)
warped = cv2.warpPerspective(undist, M, (img.shape[1],img.shape[0]))
plt.imshow(warped)

# Pipeline definition

The lane detection pipeline below does the following.
* Undistort the image using the previously calculated matrices from the calibration steps
* Apply thresholding to find the lane lines.  I combine two thresholding methods (one for white and yellow) that threshold on HLS values.
* Apply a perspective transform to create a top-down view of the lanes
* Fit a order 2 polynomial to both the left and right lane line
* Create a transparent overlay for the lane using the polynomials as sides to a polygon.

In [ ]:
def undistort(img, cameraMatrix, distCoeff):
    return cv2.undistort(img, cameraMatrix, distCoeff, None, cameraMatrix)

def threshold_white(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    h_thresh = (0, 255)
    s_thresh = (0, 255)
    l_thresh = (200, 255)
    binary = np.zeros_like(S)
    binary[(L > l_thresh[0]) & (L <= l_thresh[1])] = 1
    return binary

def threshold_yellow(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    s_thresh = (100, 255)
    l_thresh = (100, 255)
    binary = np.zeros_like(S)
    binary[(S > s_thresh[0]) & (S <= s_thresh[1]) & (L > l_thresh[0]) & (L <= l_thresh[1])] = 1
    return binary

def threshold(img):
    yellow = threshold_yellow(img)
    white = threshold_white(img)
    return np.clip(white + yellow, 0, 1)

def perspective_transform(img, M):
    return cv2.warpPerspective(img, M, (img.shape[1],img.shape[0]))

ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

def fit_poly_to_lanes(img):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(img[img.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    
    left_cr = calculate_curvature(leftx, lefty, img.shape[0])
    right_cr = calculate_curvature(rightx, righty, img.shape[0])
    

    vehicle_center_pixel = calculate_center_offset_pixel(left_fit,right_fit,img.shape[1]/2)
    pixel_offset = img.shape[1]/2 - vehicle_center_pixel
    offset_in_m = pixel_offset * xm_per_pix
    
    return(left_fit,right_fit, (left_cr + right_cr) / 2.0, offset_in_m)


def calculate_center_offset_pixel(left_fit,right_fit,y_eval):
    left = left_fit[0] * y_eval ** 2 + left_fit[1] * y_eval + left_fit[2]
    right = right_fit[0] * y_eval ** 2 + right_fit[1] * y_eval + right_fit[2]
    return (left + right) / 2
    

def calculate_curvature(x,y,y_eval):    
    fit_cr = np.polyfit(y*ym_per_pix, x*xm_per_pix, 2)
    
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    
    return curverad
    

def lane_overlay(img, Minv,left_fit,right_fit,curvature,offset):
    ploty = np.linspace(0, img.shape[0]-1, num=img.shape[0])
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw the lines on
    color_warp = np.zeros_like(img).astype(np.uint8)
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 

    # Combine the result with the original image
    
    
    overlay = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    text1 = cv2.putText(overlay,"center offset: {:10.4f} m".format(offset),(10,100), font, 2,(255,255,255),2,cv2.LINE_AA)
    text2 = cv2.putText(text1,"turn radius: {:10.4f} m".format(curvature),(10,200), font, 2,(255,255,255),2,cv2.LINE_AA)
    return text2


def pipeline(img, cameraMatrix, distCoeff, M, radii=[], offsets=[]):
    undist = undistort( img, cameraMatrix, distCoeff )
    thresholded = threshold(undist)
    persp = perspective_transform(thresholded, M)
    (left_fit,right_fit, curvature, offset) = fit_poly_to_lanes(persp)
    radii.append(curvature)
    offsets.append(offset)
    overlay = lane_overlay(undist, Minv, left_fit, right_fit,curvature,offset)    
    return overlay
    

Here we create a transparent overlay for the lane to verify that our lane detection algorithm works as expected.

In [ ]:
%matplotlib inline
test_images = glob.glob('./test_images/*.jpg')
straight_lane_lines = test_images[6]
img = plt.imread(straight_lane_lines)
lanes = pipeline(img, mtx, dist, M)
plt.imshow(lanes, cmap="gray")    

Here we apply the transparent overlay to every frame in a video recording from the vehicle.

In [ ]:
radii = []
offsets = []

def calibrated_pipeline(image):
    return pipeline(image,mtx,dist,M,radii,offsets)

from moviepy.editor import VideoFileClip
from IPython.display import HTML
clip1 = VideoFileClip("project_video.mp4")
video_output="project_video_output.mp4"
clip = clip1.fl_image(calibrated_pipeline)
%time clip.write_videofile(video_output, audio=False)
HTML("""
<video width="960" height="540" controls>
  <source src="./{0}" type="video/mp4">
</video>
""".format(video_output))
